In [196]:
!pip install beautifulsoup4

In [197]:
!pip install lxml

In [198]:
!pip install html5lib

In [236]:
from bs4 import BeautifulSoup

In [237]:
import requests

In [238]:
with open('codes.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')

In [239]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [240]:
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [241]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [242]:
Postcode = []

for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Postcode.append(cells[0].find(text=True)) #Postcode

In [243]:
Neighborhood = []
for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Neighborhood.append(cells[2].findAll(text=True))

In [244]:
Borough = []
for row in My_table.findAll("tr"): #tr is one row
    cells = row.findAll("td") #td is the input in each cell
    if len(cells) == 3: #table has three columns
        Borough.append(cells[1].find(text=True))


In [245]:
import pandas as pd
df = pd.DataFrame()
df['Postcode'] = Postcode

In [246]:
df['Borough'] = Borough

In [247]:
df['Neighborhood'] = Neighborhood

In [248]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,[Not assigned ]
1,M2A,Not assigned,[Not assigned ]
2,M3A,North York,"[Parkwoods, ]"
3,M4A,North York,"[Victoria Village, ]"
4,M5A,Downtown Toronto,"[Harbourfront, ]"


In [249]:
type(df['Neighborhood'][0]) #check the type of the values in the Neighborhood column

bs4.element.ResultSet

In [250]:
df['Neighborhood'] = df['Neighborhood'].astype(str) # convert the Neighborhood olumn to string

In [251]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,['Not assigned\n']
1,M2A,Not assigned,['Not assigned\n']
2,M3A,North York,"['Parkwoods', '\n']"
3,M4A,North York,"['Victoria Village', '\n']"
4,M5A,Downtown Toronto,"['Harbourfront', '\n']"


In [252]:
df.Neighborhood = df.Neighborhood.str[:-3] #remove unecessary part of the column

In [253]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,['Not assigned\
1,M2A,Not assigned,['Not assigned\
2,M3A,North York,"['Parkwoods', '\"
3,M4A,North York,"['Victoria Village', '\"
4,M5A,Downtown Toronto,"['Harbourfront', '\"


In [254]:
import string
df.Neighborhood = df.Neighborhood.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #further remove unecessary parts of the column

In [255]:
df.head(5)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [256]:
df['Borough'] = df['Borough'].astype(str) # convert the Borough olumn to string

In [257]:
df['Postcode'] = df['Postcode'].astype(str) # convert the Postcode column to string

In [258]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [259]:
#Remove white spaces
df['Borough'] = df['Borough'].apply(lambda x: x.strip())
df['Postcode'] = df['Postcode'].apply(lambda x: x.strip())
df.Neighborhood = df.Neighborhood.apply(lambda x: x.strip())

In [260]:
df.head(6)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [261]:
df.head(6)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [262]:
#copy data
df2 = df.copy()

In [263]:
df2 =df2.groupby('Postcode')['Neighborhood'].agg([('Neighborhood', ', '.join)]).reset_index() #GROUPING DATA BY Postcode


In [264]:
df2.loc[df2['Postcode'] == 'M5A'] #checking to see if code works

,Postcode,Neighborhood
80,M5A,"Harbourfront, Regent Park"


In [265]:
len(df2)

180

In [266]:
#merge data
df3 = df2.merge(df, left_on='Postcode', right_on='Postcode', how = 'inner')

In [267]:
df3.loc[df3['Postcode'] == 'M5A'] #checking to see if code works

,Postcode,Neighborhood_x,Borough,Neighborhood_y
118,M5A,"Harbourfront, Regent Park",Downtown Toronto,Harbourfront
119,M5A,"Harbourfront, Regent Park",Downtown Toronto,Regent Park


In [268]:
df4 = df3[~df3[['Postcode', 'Neighborhood_x']].apply(frozenset, axis=1).duplicated()] #remove duplicates

In [269]:
df4.head()

,Postcode,Neighborhood_x,Borough,Neighborhood_y
0,M1A,Not assigned,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough,Rouge
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,Highland Creek
6,M1E,"Guildwood, Morningside, West Hill",Scarborough,Guildwood
9,M1G,Woburn,Scarborough,Woburn


In [270]:
df4.loc[df4['Postcode'] == 'M5V'] #checking to see if code works

,Postcode,Neighborhood_x,Borough,Neighborhood_y
148,M5V,"CN Tower, Bathurst Quay, Island airport, Harbo...",Downtown Toronto,CN Tower


In [271]:
#remove or drop Neighborhood_y column
df4 = df4.drop(columns='Neighborhood_y')


In [272]:
#Rename Neighborhood_X
df4.rename(columns={'Neighborhood_x':'Neighborhood'}, inplace=True)

In [273]:
df4.head()

,Postcode,Neighborhood,Borough
0,M1A,Not assigned,Not assigned
1,M1B,"Rouge, Malvern",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
6,M1E,"Guildwood, Morningside, West Hill",Scarborough
9,M1G,Woburn,Scarborough


In [274]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df5 = df4.loc[df4['Borough'] != 'Not assigned'] 

In [275]:
df5.head()

,Postcode,Neighborhood,Borough
1,M1B,"Rouge, Malvern",Scarborough
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
6,M1E,"Guildwood, Morningside, West Hill",Scarborough
9,M1G,Woburn,Scarborough
10,M1H,Cedarbrae,Scarborough


In [276]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns 
#will be Queen's Park.
import numpy as np

#Add a column call Neighborhood2
df5['Neighborhood2'] = np.where(df5['Neighborhood'] == 'Not assigned', df5.Borough, df5.Neighborhood)

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [277]:
df5.head()

,Postcode,Neighborhood,Borough,Neighborhood2
1,M1B,"Rouge, Malvern",Scarborough,"Rouge, Malvern"
3,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,"Highland Creek, Rouge Hill, Port Union"
6,M1E,"Guildwood, Morningside, West Hill",Scarborough,"Guildwood, Morningside, West Hill"
9,M1G,Woburn,Scarborough,Woburn
10,M1H,Cedarbrae,Scarborough,Cedarbrae


In [278]:
# check to see if it works
df5.loc[df5['Neighborhood'] == 'Not assigned'] #checking to see if code works

,Postcode,Neighborhood,Borough,Neighborhood2
194,M7A,Not assigned,Queen's Park,Queen's Park


In [279]:
# Now we can drop the Neighborhood column
df5 = df5.drop(columns='Neighborhood')

In [280]:
# Rename the Neighborhood2 coloumn back as Neighborhood
df5.rename(columns={'Neighborhood2':'Neighborhood'}, inplace=True)

In [281]:
#Rename final data
finaldata  = df5

In [282]:
#checking the shape of the finaldata
finaldata.shape

(103, 3)

In [283]:
finaldata.head()

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge, Malvern"
3,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
6,M1E,Scarborough,"Guildwood, Morningside, West Hill"
9,M1G,Scarborough,Woburn
10,M1H,Scarborough,Cedarbrae


In [284]:
#Part 2

In [285]:
#Download Geopatial Data
geodata = pd.read_csv('http://cocl.us/Geospatial_data')

In [286]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [287]:
#merge geospatial data with the final data and name full data data
#merge data
fulldata = finaldata.merge(geodata, left_on='Postcode', right_on='Postal Code', how = 'inner')

In [288]:
fulldata.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [289]:
#drop Postal Code column
fulldata = fulldata.drop(columns='Postal Code')


In [290]:
fulldata.head(3)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711


In [291]:
## Explore and cluster the neighborhoods in Toronto.


In [292]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(fulldata['Borough'].unique()),
        fulldata.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [293]:
#There are 11 boroughs and 103 neighborhoods in the dataset

In [294]:
#Use geopy library to get the latitude and longitude values of New York City.
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address) #this gets the latitude and longitude coordinates of the address specified above
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [295]:
%matplotlib inline

In [296]:
#Create a map of New York with neighborhoods superimposed on top.
# create map of New York using latitude and longitude values
#Note that here we use the full data.
import folium # map rendering library
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(fulldata['Latitude'], fulldata['Longitude'], fulldata['Borough'], fulldata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [297]:
#Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#Define Foursquare Credentials and Version

CLIENT_ID = 'GP44JHK2IBCMB4UPT552AURDDF24LK12K3MVEOIMWZX0E1D0' # your Foursquare ID
CLIENT_SECRET = 'WTXVRIFXJMKAW45ZETIKJOWOES0D5OQ4GYGQQSV5H5V41DZC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GP44JHK2IBCMB4UPT552AURDDF24LK12K3MVEOIMWZX0E1D0
CLIENT_SECRET:WTXVRIFXJMKAW45ZETIKJOWOES0D5OQ4GYGQQSV5H5V41DZC


In [298]:
#Let's explore the first neighborhood in our dataframe.
#Get some neighborhood's name, say Lawrence Park.


In [299]:
fulldata.loc[fulldata['Neighborhood'] == 'Lawrence Park']

,Postcode,Borough,Neighborhood,Latitude,Longitude
44,M4N,Central Toronto,Lawrence Park,43.72802,-79.38879


In [300]:
#We see that the index of this row is 44, so we run the following code to select the neighborhood name

In [301]:
fulldata.loc[44, 'Neighborhood']

'Lawrence Park'

In [302]:
#Get the neighborhood's latitude and longitude values.

In [303]:
neighborhood_latitude = fulldata.loc[44, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = fulldata.loc[44, 'Longitude'] # neighborhood longitude value

neighborhood_name = fulldata.loc[44, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


In [304]:
#Now, let's get the top 100 venues that are in Lawrence Park within a radius of 500 meters.
#First, let's create the GET request URL. Name your URL url.
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GP44JHK2IBCMB4UPT552AURDDF24LK12K3MVEOIMWZX0E1D0&client_secret=WTXVRIFXJMKAW45ZETIKJOWOES0D5OQ4GYGQQSV5H5V41DZC&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [305]:
#Send the GET request and examine the resutls

In [306]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0229cf6f0aa2002c9f9633'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

In [307]:
#From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, 
#let's borrow the get_category_type function from the Foursquare lab.

In [308]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [309]:
#Now we are ready to clean the json and structure it into a pandas dataframe.

In [310]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [311]:
print(len(nearby_venues), 'venues are close to Lawrence Park')

3 venues are close to Lawrence Park


In [312]:
#And how many venues were returned by Foursquare?

In [313]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


In [314]:
#2. Explore Neighborhoods in Toronto
#Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [315]:
#Repeat the process for all neiborhoods, not just marble hills
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [316]:
#run the above function on each neighborhood and create a new dataframe called manhattan_venues.

In [317]:
# type your answer here

toronto_venues = getNearbyVenues(names=fulldata['Neighborhood'],
                                   latitudes=fulldata['Latitude'],
                                   longitudes=fulldata['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam OShanter
Agincourt North, LAmoreaux East, Milliken, Steeles East
LAmoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riv

In [318]:
#Let's check the size of the resulting dataframe¶

In [319]:
print(toronto_venues.shape)
toronto_venues.head()

(2254, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


In [320]:
#Let's check how many venues were returned for each neighborhood

In [321]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, LAmoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55


In [322]:
#Let's find out how many unique venue categories can be curated from all the returned venues¶

In [323]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 279 uniques categories.


In [324]:
#3. Analyze Each Neighborhood

In [325]:
# one hot encoding just for the venue categoty column
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [326]:
#And let's examine the new dataframe size.

In [327]:
toronto_onehot.shape

(2254, 279)

In [328]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [329]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, LAmoreaux East, Milliken, Ste...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [330]:
#Let's confirm the new size

In [331]:
toronto_grouped.shape

(100, 279)

In [332]:
#Let's print each neighborhood along with the top 5 most common venues¶

In [333]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.05
1                 Café  0.05
2                  Bar  0.04
3  American Restaurant  0.04
4           Steakhouse  0.04


----Agincourt----
                venue  freq
0      Sandwich Place  0.25
1              Lounge  0.25
2      Breakfast Spot  0.25
3  Chinese Restaurant  0.25
4         Yoga Studio  0.00


----Agincourt North, LAmoreaux East, Milliken, Steeles East----
                venue  freq
0          Playground  0.33
1    Asian Restaurant  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
            venue  freq
0   Grocery Store  0.18
1        Pharmacy  0.09
2      Beer Store  0.09
3    Liquor Store  0.09
4  Sandwich Place  0.09


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place   0.2
1    Skating Rink   0.1
2 

In [334]:
#Let's put that into a pandas dataframe
#First, let's write a function to sort the venues in descending order.

In [335]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [336]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [337]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Bar,American Restaurant,Steakhouse,Thai Restaurant,Gym,Bakery,Hotel,Burger Joint
1,Agincourt,Chinese Restaurant,Lounge,Sandwich Place,Breakfast Spot,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
2,"Agincourt North, LAmoreaux East, Milliken, Ste...",Park,Asian Restaurant,Playground,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Liquor Store,Pharmacy,Pizza Place,Sandwich Place,Coffee Shop,Video Store
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Skating Rink,Pharmacy,Pub,Dance Studio,Pool,Sandwich Place,Women's Store


In [338]:
#4. Cluster Neighborhoods
#Run k-means to cluster the neighborhood into 5 clusters.

In [339]:
# set number of clusters
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1) #drop the neibaorhood column

In [340]:
toronto_grouped_clustering.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.01,0.0,0.0
1,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
2,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0
3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.090909,0.0,0.0,0.00,0.0,0.0
4,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0


In [341]:
# run k-means clustering
# import k-means from clustering stage
from sklearn.cluster import KMeans

kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] #first 10 cluster labels

array([4, 4, 2, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [342]:
#merge cluster data to previous data
# add clustering labels
toronto_data = fulldata
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [349]:
toronto_merged.head(5) # first five rows

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,4.0,Fast Food Restaurant,Print Shop,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Bar,Construction & Landscaping,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Medical Center,Electronics Store,Spa,Pizza Place,Intersection,Tech Startup,Breakfast Spot,Rental Car Location,Mexican Restaurant,Discount Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,Convenience Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Athletics & Sports,Thai Restaurant,Bank,Bakery,Fried Chicken Joint,Caribbean Restaurant,Lounge,Hakka Restaurant,Cosmetics Shop,Costume Shop


In [350]:
toronto_merged['Cluster Labels'].unique()

array([ 4.,  2., nan,  1.,  0.,  3.])

In [351]:
# Total number of missing values
print (toronto_merged.isnull().sum().sum())

33


In [353]:
#count number of missing values in each column
toronto_merged.isna().sum()

Postcode                  0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            3
1st Most Common Venue     3
2nd Most Common Venue     3
3rd Most Common Venue     3
4th Most Common Venue     3
5th Most Common Venue     3
6th Most Common Venue     3
7th Most Common Venue     3
8th Most Common Venue     3
9th Most Common Venue     3
10th Most Common Venue    3
dtype: int64

In [362]:
#drop rows with mising values
#toronto_merged.dropna()
toronto_merged=toronto_merged.dropna()

In [363]:
# check again for missing values
toronto_merged.isna().sum()

Postcode                  0
Borough                   0
Neighborhood              0
Latitude                  0
Longitude                 0
Cluster Labels            0
1st Most Common Venue     0
2nd Most Common Venue     0
3rd Most Common Venue     0
4th Most Common Venue     0
5th Most Common Venue     0
6th Most Common Venue     0
7th Most Common Venue     0
8th Most Common Venue     0
9th Most Common Venue     0
10th Most Common Venue    0
dtype: int64

In [364]:
toronto_merged['Cluster Labels'].unique()

array([4., 2., 1., 0., 3.])

## Dont forget to add markdowns to explain your work